In [164]:
import numpy as np
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
import re
import matplotlib
from spectral import *
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.size'] = 3
matplotlib.rcParams['lines.linewidth'] = 0.5
matplotlib.rcParams['axes.linewidth']= 0.5
matplotlib.rcParams['xtick.major.width'] = 0.5
from utils import *
import cv2
from tqdm import tqdm

# Set the font family to Arial
plt.rc('font', family='Arial')

plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.size'] = 7

In [ ]:
from datetime import date
today = date.today()
date_str = today.strftime('%d%b%Y')
print ('Date prefix:', date_str)

In [ ]:
SAVEDIR = 'cv2_analysis'
if not os.path.isdir(SAVEDIR):
    os.mkdir(SAVEDIR)
    print ('Made', SAVEDIR)

In [167]:
def classify_with_unmixing (img, endmembers, reference_spectrum):
    img_flattened= np.reshape(img, (img.shape[0]*img.shape[1], img.shape[2]))
    img_flattened = img_flattened / np.nanmax(img_flattened, axis=1, keepdims=True)
#     img_flattened[img_flattened==0] = np.nan
    
    mat = np.vstack([-reference_spectrum, endmembers])

    #fit
    scored_img = UCLS(img_flattened, mat)
    scored_img[scored_img<0] = 0
    scored_img  = np.reshape(scored_img[:,0], img.shape[:2])
    return scored_img



def sort_grid_list(points, tolerance=20):
    import numpy as np
    
    points = np.array(points)
    points_sorted_by_y = points[np.argsort(points[:, 1])]  # sort by y
    
    rows = []
    current_row = [points_sorted_by_y[0]]
    
    for pt in points_sorted_by_y[1:]:
        if abs(pt[1] - current_row[-1][1]) < tolerance:
            current_row.append(pt)
        else:
            rows.append(current_row)
            current_row = [pt]
    rows.append(current_row)

    # Sort each row by x (left to right)
    sorted_points = [pt for row in rows for pt in sorted(row, key=lambda x: int(x[0]))]
    return sorted_points


In [168]:
lib = envi.open('data/input/087_ELOP Eden Fleshler/results/REFLECTANCE_087.hdr')
hsi_img = np.array(lib.load())

In [ ]:
# reference_file = np.load('../absorbance_data/bpHO-smURFP_infered_absorbance_from_pellets_30May2024.npy')
# reference_file = np.load('../hsi_detect/absorbance_data/YF10_infered_absorbance_from_MAFATJul2023_low_flight.npy')

reference_file = np.load('data/input/YF10_infered_absorbance_from_pellets_09Jul2024.npy')

reference_wls = reference_file[0,:]
reference_spec = reference_file[1,:] #*.1

reference_spec = np.interp(lib.bands.centers, reference_wls, reference_spec)
plt.plot(lib.bands.centers, reference_spec)

In [ ]:
filt_endmembers, clust_ls = kmeans_hierarchical_extract_endmembers(hsi_img, 
                                                         return_cluster_idxs=True, 
                                                         reference_spec=reference_spec, reduced_dims=3,
                                                         filter_threshold=0.9)

# cluster_based_extract_endmembers(hsi_img,10)
# filt_endmembers = filter_endmembers(endmembers_gradient_A_B, reference_spec)

scored_gradient = classify_with_unmixing(hsi_img, filt_endmembers[0], reference_spec)
scored_gradient[scored_gradient<0] = 0


In [ ]:
plt.figure()
plt.imshow(scored_gradient, vmax=0.8, cmap='inferno')
plt.xticks([])
plt.yticks([])
plt.box(False)
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(1-hsi_img[:,:,np.argmax(reference_spec)],  cmap='inferno', vmin=0, vmax=1)
plt.xticks([])
plt.yticks([])
plt.box(False)
plt.colorbar()
print (lib.bands.centers[np.argmax(reference_spec)])
plt.show()

In [ ]:
plt.figure()
rgb_img = bandpass_rgb_function(hsi_img, lib.bands.centers, coeffs=[2.5,2.5,2.5])
plt.imshow(rgb_img)
plt.xticks([])
plt.yticks([])
plt.box(False)
plt.show()

In [ ]:
plt.figure()
plt.imshow(hsi_img[:,:,np.argmax(reference_spec)]/np.trapz(hsi_img, axis=2),  cmap='inferno')
plt.xticks([])
plt.yticks([])
plt.box(False)
plt.colorbar()
print (lib.bands.centers[np.argmax(reference_spec)])
plt.show()

In [175]:
def mask_ellipse(array, center, radius_x, radius_y):
    """
    Masks out an ellipse within a NumPy array.
    
    Parameters:
        array (numpy.ndarray): The input array.
        center (tuple): The center coordinates of the ellipse (row, column).
        radius_x (int): The radius of the ellipse along the x-axis.
        radius_y (int): The radius of the ellipse along the y-axis.
    
    Returns:
        numpy.ndarray: The masked array with the ellipse.
    """
    mask = np.zeros_like(array, dtype=bool)
    rows, cols = array.shape[:2]
    cx, cy = center
    
    y, x = np.ogrid[:rows, :cols]
    mask[((x - cx) / radius_x) ** 2 + ((y - cy) / radius_y) ** 2 <= 1] = True
    
    masked_array = array*mask
    return masked_array

def make_ellipse_masks(mask_shape, centers, radii):
    """
    Creates a mask for an ellipse of a given radius and center.
    radii is a list of tuples defining the x and y dimensions of the ellipse.
    """
    mask = np.zeros(mask_shape[:2])
    for c, r in zip(centers, radii):
        mask += mask_ellipse(np.ones_like(mask), c, r[0], r[1])
    return mask


def make_circle_mask  (mask_shape, centers, radii):
    """
    Creates a mask for a circle of a given radius and center.
    """
    mask = np.zeros(mask_shape[:2])
    for c, r in zip(centers, radii):
        mask += mask_ellipse(np.ones_like(mask), c, r, r)
    return mask


In [ ]:
# To identify the repeating shape, we need to create a "mask" of the same shape as the objects
# Let's crop the image to get a sense for the dimensions

MIN_X = 215
MIN_Y = 385
MAX_X = 264
MAX_Y = 445
plt.imshow(rgb_img[MIN_Y:MAX_Y, MIN_X:MAX_X])

In [ ]:

GRAD_RADIUS_X = int((MAX_X - MIN_X)/2)
GRAD_RADIUS_Y = int((MAX_Y - MIN_Y)/2)
            
# centers  = [[590,400], [1250,400], [880,1000]]
# radii = [310, 310, 310]

grad_centers  = [[GRAD_RADIUS_X,GRAD_RADIUS_Y]]
grad_radii = [(GRAD_RADIUS_X, GRAD_RADIUS_Y)]
grad_im_size = (2*grad_radii[0][1], 2*grad_radii[0][0])
# np.array(data_dict['round']['t0']['image']).shape
grad_template_mask = ((1-make_ellipse_masks(grad_im_size, grad_centers, grad_radii))*254).astype(np.uint8)
plt.imshow(rgb_img[MIN_Y:MAX_Y, MIN_X:MAX_X])
plt.imshow(grad_template_mask, alpha=0.3)

In [ ]:
lib.bands.centers[170]

In [ ]:
# Make masks to get signal from multiple experiments with plates of the same size
BINARY_THRESHOLD = 0.56 # the threshold for the binary mask 
BANDS = [690, 900]

# Could include > 1 image
grad_img_ls = [hsi_img]
plate_positions = []


# convert images to gray scale for identification
grad_img_gray_ls = []
band_idx_1 = np.argmin(np.abs(np.array(lib.bands.centers) - BANDS[0]))
band_idx_2 = np.argmin(np.abs(np.array(lib.bands.centers) - BANDS[1]))
for h_img in grad_img_ls:
    ratio_img = h_img[:,:,band_idx_1]/h_img[:,:,band_idx_2]
    ratio_img = (ratio_img-np.min(ratio_img)) / (np.max(ratio_img)-np.min(ratio_img))
    grad_img_gray_ls.append((ratio_img*254).astype(np.uint8))

# # Apply template matching
matching_results = []
matching_stats = []
grad_mask_ls = []
for gray in grad_img_gray_ls:
    plt.figure()
    plt.title(f'Gray image from band ratio {lib.bands.centers[band_idx_1]} / {lib.bands.centers[band_idx_2]}')
    plt.imshow(gray)
    plt.show()
    result = cv2.matchTemplate(gray, grad_template_mask, cv2.TM_CCOEFF_NORMED)
    plt.figure()
    plt.title('Map of template shape matching, thresholded')
    plt.imshow(gray, cmap='Greys')
    plt.imshow(np.pad(result, [[GRAD_RADIUS_Y,GRAD_RADIUS_Y], [GRAD_RADIUS_X,GRAD_RADIUS_X]]), alpha=0.4)
    plt.show()
    # Threshold the result
    _, result = cv2.threshold(result, BINARY_THRESHOLD, 1, cv2.THRESH_BINARY)
    result = (np.array(result)*254).astype(np.uint8)
    result = np.pad(result, [[GRAD_RADIUS_Y,GRAD_RADIUS_Y], [GRAD_RADIUS_X,GRAD_RADIUS_X]])
    matching_results.append(result)
    
    _, _, stats, centroids = cv2.connectedComponentsWithStats(result, connectivity=8)

    grad_found_centers = [c for c,s in zip(centroids[1:], stats[1:]) if s[4]>10]
    grad_found_centers = sort_grid_list(grad_found_centers, tolerance=25)
    plate_positions.append(grad_found_centers)
    plt.figure()
    plt.title('Map of template shape matching')
    plt.imshow(gray, cmap='Greys')
    plt.imshow(result, alpha=0.4)
    plt.show()
    
    grad_masks = []
    for c in grad_found_centers:
        grad_masks.append(make_ellipse_masks(np.array(gray).shape, [c], [(GRAD_RADIUS_X, GRAD_RADIUS_Y)]))
    
    grad_mask_ls.append(grad_masks)
    plt.figure()
    plt.imshow(gray, cmap='Greys')

    tab20 = plt.get_cmap('tab20')
    for idx, (m,c) in enumerate(zip(grad_masks,grad_found_centers)):
        m[m==0] = np.nan
        m = m[:,:,np.newaxis] * np.array(tab20(idx))
        plt.imshow(m, alpha=0.5,)
        plt.text(x=c[0],y=c[1],s=str(idx))
    plt.show()

    
    
    
# #     for i in range(3)


In [ ]:
import pandas as pd
conc_map = pd.read_csv('data/input/087_ELOP Eden Fleshler/results/concentration_map.csv', header=None).values
print(conc_map)

In [ ]:

# function to aggregate the scores across a plate
score_fxn = lambda x: np.nanmean(x) 


grad_scores_ls = []
grad_abs_ls = []
grad_norm_abs_ls = []


for grad_masks, grad_scored, img in zip(grad_mask_ls, [scored_gradient], grad_img_ls):
    single_wl_img = img[:,:,np.argmax(reference_spec)]
    img_sum = np.sum(img, axis=2)
    grad_scores = []
    grad_abs = []
    grad_norm_abs = []
    for mask in tqdm(grad_masks):
        grad_scores.append(score_fxn(mask*grad_scored))
        grad_abs.append(1-score_fxn(mask*single_wl_img))
        grad_norm_abs.append(1-score_fxn(mask*single_wl_img/img_sum))
    grad_scores_ls.append(grad_scores)
    grad_abs_ls.append(grad_abs)
    grad_norm_abs_ls.append(grad_norm_abs)

In [ ]:
# Flatten the concentration map into a single vector
grad_concs = np.array([x for y in conc_map for x in y])
grad_mean_classification = np.array(grad_scores_ls[0])
grad_concs = np.array(grad_concs)[~np.isnan(grad_concs)]

data_df = pd.DataFrame({'pC-HSL (nM)':np.array(grad_concs), 'Classification Score (mean)':np.array(grad_mean_classification)})

print (f'{SAVEDIR}/{date_str}sensing_on_sand_hill_plot.pdf')
plt.scatter(np.array(grad_concs), np.array(grad_mean_classification))

In [ ]:
### UNCOMMENT TO SAVE DATA
savepath = f'{SAVEDIR}/{date_str}_data.csv'
print (f'data saved to {savepath}')
data_df.to_csv(savepath)